# Praktikum 1 : Klasifikasi 🐱 dan 🐶

## Langkah 1 - Import Library

In [1]:
# Mount Google Drive agar dapat mengakses dataset yang disimpan di Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Import TensorFlow dan modul ImageDataGenerator dari Keras untuk augmentasi gambar
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

# Langkah 2 - Pra Pengolahan Data
## Langkah 2.1 Pra pengolahan data training

In [3]:
# Membuat objek ImageDataGenerator untuk augmentasi data pelatihan
# Rescale dilakukan untuk mengubah nilai piksel menjadi rentang 0-1
# augmentasi dilakukan dengan shear, zoom, dan horizontal flip.
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Menggunakan objek train_datagen untuk memuat dan mengaugmentasi data pelatihan
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')
# Data pelatihan diambil dari direktori 'training_set', dengan target size 64x64,
# batch size sebesar 32, dan mode kelas binary (klasifikasi biner: kucing atau anjing).

Found 8000 images belonging to 2 classes.


## Langkah 2.2 Pra Pengolahan Data Testing

In [4]:
# Membuat objek ImageDataGenerator untuk normalisasi data uji
test_datagen = ImageDataGenerator(rescale = 1./255)

# Menggunakan objek test_datagen untuk memuat dan normalisasi data uji
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')
# Data uji diambil dari direktori 'test_set', dengan target size 64x64,
# batch size sebesar 32, dan mode kelas binary (klasifikasi biner: kucing atau anjing).

Found 2000 images belonging to 2 classes.


# Langkah 3 - Pembuatan Model CNN
## Langkah 3.1.  - Inisiasi Model CNN

In [5]:
cnn = tf.keras.models.Sequential() # Membuat model CNN dengan menggunakan Sequential()

## Langkah 3.2. - Pembuatan Layer Konvolusi 1

In [6]:
# Menambahkan layer Conv2D pertama dengan 32 filter, kernel size 3x3, fungsi aktivasi ReLU, dan input shape 64x64x3
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

## Langkah 3.3 - Pembuatan Layer Pooling 1

In [7]:
# Menambahkan layer MaxPooling2D dengan pool size 2x2 dan strides 2
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

## Langkah 3.4 - Pembuatan Layer Konvolusi 2 dan Pooling 2

In [8]:
# Menambahkan layer Conv2D kedua dengan 32 filter, kernel size 3x3, dan fungsi aktivasi ReLU
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
# Menambahkan layer MaxPooling2D dengan pool size 2x2 dan strides 2
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

## Langkah 3.5 - Flattening

In [9]:
# Menambahkan layer Flatten untuk meratakan output menjadi vektor
cnn.add(tf.keras.layers.Flatten())

## Langkah 3.6 - Fully Connected Layer 1 (Input)

In [10]:
# Menambahkan layer Dense dengan 128 neuron dan fungsi aktivasi ReLU
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

## Langkah 3.7 - Fully Connected Layer 2 (Output)

In [11]:
# Menambahkan layer Dense output dengan 1 neuron dan fungsi aktivasi sigmoid (karena ini adalah masalah klasifikasi biner)
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Langkah 3.8 - Compile Model CNN

In [12]:
# Meng-compile model dengan optimizer 'adam', loss function 'binary_crossentropy', dan metrik akurasi
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Langkah 4 - Fit CNN

In [13]:
# Melakukan pelatihan model pada data pelatihan dan evaluasi pada data validasi selama 25 epoch
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 1645s 7s/step - loss: 0.6568 - accuracy: 0.5968 - val_loss: 0.5985 - val_accuracy: 0.6805
Epoch 2/25
250/250 [==============================] - 38s 150ms/step - loss: 0.5895 - accuracy: 0.6837 - val_loss: 0.5400 - val_accuracy: 0.7290
Epoch 3/25
250/250 [==============================] - 38s 153ms/step - loss: 0.5397 - accuracy: 0.7181 - val_loss: 0.5553 - val_accuracy: 0.7230
Epoch 4/25
250/250 [==============================] - 43s 171ms/step - loss: 0.5128 - accuracy: 0.7430 - val_loss: 0.5292 - val_accuracy: 0.7425
Epoch 5/25
250/250 [==============================] - 37s 147ms/step - loss: 0.4913 - accuracy: 0.7629 - val_loss: 0.5023 - val_accuracy: 0.7555
Epoch 6/25
250/250 [==============================] - 37s 148ms/step - loss: 0.4819 - accuracy: 0.7679 - val_loss: 0.5074 - val_accuracy: 0.7670
Epoch 7/25
250/250 [==============================] - 37s 150ms/step - loss: 0.4559 - accuracy: 0.7809 - val_loss: 0.4586 - val_acc

# Langkah 5 - Prediksi dengan 1 Citra

In [15]:
# Melakukan prediksi untuk satu gambar
import numpy as np
from keras.preprocessing import image

# Memuat gambar uji dengan ukuran target 64x64
test_image = image.load_img('/content/drive/MyDrive/dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
# Mengubah gambar menjadi array numpy
test_image = image.img_to_array(test_image)
# Menambahkan dimensi baru pada array (menjadi tensor) untuk sesuai dengan input model
test_image = np.expand_dims(test_image, axis = 0)
# Melakukan prediksi menggunakan model CNN
result = cnn.predict(test_image)

# Mendapatkan kelas yang sesuai dari hasil prediksi
# (Catatan: Angka 1 mewakili 'dog', dan angka 0 mewakili 'cat' sesuai dengan setting model)
training_set.class_indices

# Menentukan hasil prediksi berdasarkan nilai yang didapatkan
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 [==============================] - 0s 150ms/step
